In [ ]:
%pip install scikit-learn
%pip install joblib
%pip install numpy
%pip install scikit-image

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog

# Settings
image_size = (32, 32)
data_dir = './datasets/ProcessedFruitDataset/'

data, labels, class_names = [], [], []

# loading images from our folders manually
for label, class_name in enumerate(sorted(os.listdir(data_dir))):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        class_names.append(class_name)
        for filename in os.listdir(class_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(class_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, image_size)

                features = hog(img, 
                orientations=9, 
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), 
                block_norm='L2-Hys')
                
                data.append(features)
                labels.append(label)

# turning our arrays into numpy arrays for performance
data = np.array(data)
labels = np.array(labels)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# Split data
TrainData, TestData, TrainLabels, TestLabels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
# Train SVM
clf = svm.SVC(kernel='linear')
clf.fit(TrainData, TrainLabels)

In [ ]:
from sklearn.metrics import accuracy_score

# Test accuracy
TestPredictions = clf.predict(TestData)
print("Accuracy:", accuracy_score(TestLabels, TestPredictions))

In [ ]:
import joblib
# Save model and classes
joblib.dump(clf, './models/svm_model.joblib')
joblib.dump(class_names, './models/class_names.joblib')